In [ ]:
# pip install -e git+https://github.com/tomquirk/realestate-com-au-api.git#egg=realestate_com_au_api


In [16]:
from realestate_com_au import RealestateComAu

api = RealestateComAu()

# Get property listings
# listings = api.search(locations=["seventeen seventy, qld 4677"], channel="buy", keywords=["tenant"], exclude_keywords=["pool"])



In [6]:
import pandas as pd

# https://www.matthewproctor.com/australian_postcodes
df_postcode = pd.read_excel (r'/Users/joezhou/Downloads/Realestate/australian_postcodes.xlsx', sheet_name='Sheet1')

# sample test suburb
df_postcode_subset = df_postcode.loc[(df_postcode['Locality']=='LIDCOMBE') & (df_postcode['Postcode']==2141)]

df_postcode_subset["VAR_Find"] = '["'+df_postcode_subset['Postcode'].apply(str)+","+df_postcode_subset['Locality']+'"]'

list_postcode = list(df_postcode_subset['VAR_Find'].dropna())


In [6]:
# listings_rent = api.search(locations=["ACT, NT, SA, WA, NSW, QLD, VIC, TAS"], channel="rent")
# listings_all = api.search(locations=["ACT, NT, SA, WA, NSW, QLD, VIC, TAS"])

# last run for rent was 7 mins
#note, output caps out at 2,000 records

In [ ]:
#  Find_Text = ["2141,LIDCOMBE"]

#  df_out = api.search(locations=Find_Text)
# 870 search records with single quotes

In [42]:
# try a loop approach by suburb

DF_RENT = pd.DataFrame()
DF_ALL = pd.DataFrame()

def REALT_INFO(Find_Text):    
    global DF_RENT
    try:
        df_imp_rent = api.search(locations=Find_Text, channel="rent")
        df_imp_all = api.search(locations=Find_Text)

        DF_RENT = DF_RENT.append(df_imp_rent, ignore_index=True) 
        DF_ALL = DF_ALL.append(df_imp_all, ignore_index=True) 

        print("found for:",Find_Text)
        
    except Exception:
        print("Error with:",Find_Text)
        pass
    
    return()

# Download all
for Find_Var in list_postcode:
   REALT_INFO(Find_Var)


found for: ["2141,LIDCOMBE"]


In [8]:


# RAW_DATA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_REALESTATE_ALL.pkl') 

RAW_DATA_SUBSET = DF_ALL.drop(columns=['images', 'images_floorplans', 'listers'])

# summarise by postcode

RAW_DATA_SUM = RAW_DATA_SUBSET.groupby("postcode")["price"].median()
RAW_DATA_SUM = pd.DataFrame(RAW_DATA_SUM)
RAW_DATA_SUM.rename(columns={'price':'price_BUY'}, inplace = True)


In [5]:
# RAW_DATA_RENT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_REALESTATE_RENT.pkl') 

RAW_DATA_RENT_SUBSET = DF_RENT.drop(columns=['images', 'images_floorplans', 'listers'])

# summarise by postcode

RAW_DATA_RENT_SUM = RAW_DATA_RENT_SUBSET.groupby("postcode")["price"].median()
RAW_DATA_RENT_SUM = pd.DataFrame(RAW_DATA_RENT_SUM)
RAW_DATA_RENT_SUM.rename(columns={"price": "price_RENT"}, inplace = True)


In [9]:
df_comp = RAW_DATA_SUM.merge(RAW_DATA_RENT_SUM,how='left',on='postcode')

df_comp['YEARLY_RENT']=df_comp['price_RENT']*52
df_comp['RENT_YIELD']=df_comp['YEARLY_RENT']/df_comp['price_BUY']

# 7303

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
